In [1]:
import os
import sys
import json
from kaggle_secrets import UserSecretsClient
from google.oauth2 import service_account
os.chdir('/kaggle/input/patent-art')
sys.path.append('/kaggle/input/patent-art')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/patent-art'

In [ ]:
!uv pip install --no-cache-dir -r requirements-kaggle.txt &> /dev/null

In [ ]:
def get_credentials(secret_name: str = "gcp_service_account")  -> service_account.Credentials:
    """Fetch GCP Credentials"""
    user_secrets =  UserSecretsClient()
    service_account_json = user_secrets.get_secret(secret_name)

    with open("/tmp/service_account.json", "w") as f:
        f.write(service_account_json)
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/tmp/service_account.json"
    credentials_info = json.loads(service_account_json)
    credentials = service_account.Credentials.from_service_account_info(credentials_info)


    gcp_secrets = json.loads(user_secrets.get_secret(secret_name))


    os.environ["project_id"] = gcp_secrets["project_id"]
    os.environ["dataset_id"] = user_secrets.get_secret("dataset_id") 
    os.environ["publication_table"] = user_secrets.get_secret("publication_table")
    os.environ["small_model_id"] = user_secrets.get_secret("small_model_id")
    os.environ["embedding_table"] = user_secrets.get_secret("embedding_table")
    os.environ["service_account_path"] = "/tmp/service_account.json"
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/tmp/service_account.json"

    os.environ["hf_token"] = user_secrets.get_secret('hf_token') 

    return user_secrets, credentials

user_secrets, credentials = get_credentials()

In [ ]:
from src.kaggle.patent_dashboard_chart_functions import (
    create_patent_dashboard_demo
)
from run_patent_search_pipeline import (
    run_semantic_search_pipeline,
)
from src.kaggle.kaggle_patent_search_demo import(
    demo_search_interface
)
from src.kaggle.kaggle_patent_chart_metrics import(
    create_latency_visualization,
    create_bigquery_visualization,
    create_discoverability_visualization,

)

<h1>Patent Intelligence: Semantic Search for Innovation Discovery</h1>

## Problem Statement

The exponential growth in global patent publication (3.6M applications in 2023, representing 2.7% growth year-over-year [WIPO 2024](https://www.wipo.int/edocs/pubdocs/en/wipo-pub-943-2024-en-wipo-ip-facts-and-figures-2024.pdf). 68.7% of global patent aplications in 2023 originated from China (WIPO 2024) displaying China's dominance. Globally, patent filings increased by 18.1% between 2019-2023 indicating accelerating innovation; this creates significant challenges for innovation research and competitive intelligence. Our analysis of Google's patent publication corpus reveals that English-language patent publication peaked in 2020-2021, declined 5.6% through 2023, with partial recovery in 2024.<br>
Traditional keyword-based patent search systems fail to capture semantic relationships between technologies, forcing researchers to spend excessive time on manual review and potentially missing relevant prior art. With 2.9M English-language patents from our 2024 dataset(14.4GB) representing the first half of the year of global innovation output, there is a clear need for semantic search capabilities that can understand technological concepts beyond surface-level keyword understanding.

## Architectural Diagram & Decisions

**Hybrid Processing Strategy:** The combination of performance constraints and platform limitations led to a multi-stage architecture:
1. Offline Bulk Processing: Kaggle GPU for embedding generation (6.5 hours for 2.9M patents)
2. Real-time Query Processing: CPU-based query embeddings (~500ms per query)
3. Production-Ready Optimization: BigQuery partitioning and clustering for cost efficiency
4. Edition-Aware Design: Index-ready architecture that can utilize vector indexing when deployed on Enterprise+ tiers
5. Streamlit Cloud Demo - Easily shareable interactive dashboard enabling non-technical stakeholders to experience semantic patent discovery with explainability, leveraging real-time BigQuery AI integration for enterprise adoption validation

## Data Pipeline Overview

This notebook demonstrates the core BigQuery AI semantic search functionality. 
The complete pipeline involved:
1. Raw dataset filtering (2.6TB → 49M patents → 2.9M subset)
2. Embedding generation (6.5 hours on Kaggle GPU)
3. BigQuery AI vector search implementation (shown below)

Due to compute costs, the preprocessing steps were executed offline. Check the <a id="Assets"></a>[Repo](#Assets) for source code

## Static Demo: Semantic Patent Search with Explainability

**Text Query Mode:** Users enter natural language descriptions of technical concepts or inventions. The system provides configurable search parameters:

- **Date Range Filter:** Limited to our 2024 dataset (January-June 2024, 2.9M patents)
- **Results Slider:** Select top-K similar patents (1-20 results)

**Patent Number Mode:** Direct lookup by entering specific patent publication numbers (one per line). If multiple patent numbers
are provided, then the system computes the average value of the embeddings to return similar patents
**Search Results & Relevance Scoring:**
Upon executing a search, the system returns ranked results with comprehensive metadata:

- Patent title and publication details (number, date, country)
-  **Document Relevance Score:** Semantic similarity percentage

    - **High Relevance:** >65% (strong conceptual match)
    - **Moderate Relevance:** 50-65% (related concepts)
    - **Low Relevance:** <50% (weak conceptual connection)


- Complete patent abstract for context

**Input Validation & Quality Control:**
The interface implements query sanitization and validation to optimize database performance. While most invalid queries are filtered, some may pass validation but return low-relevance results due to poor semantic matching.<br>
**Explainability Feature:**
Our advanced explainability component provides transparency into search results by identifying the most semantically similar sentences between the user query and patent content. This feature returns up to 3 explanatory results ranked by sentence-level similarity scores, enabling users to understand why specific patents were recommended.
This approach demonstrates BigQuery AI's capability to provide not just semantic search results, but interpretable insights into the matching process.



In [ ]:
demo_search_interface()

## BigQuery AI Semantic Search: Technical Implementation and Scalability Analysis

The following sections present comprehensive performance analysis of our BigQuery AI implementation, demonstrating sub-4 second query times across 2.9M patents, 84% cost reduction and 12% time reduction through intelligent partitioning, and 98.3% unique discovery rates compared to keyword search methods.<br>
Testing was carried out across 10 diverse technology queries avoiding caching, comparing performances between development (laptop) and cloud (Kaggle) environments. Each query returned 20 semantic results including full patent metadata (title, abstract, publication details, similarity scores), representing typical user search behavior:

#### Scalability Validation

Our implementation demonstrates production-ready scalability across multiple dimensions:

**Data Volume Scalability:**
- Successfully processes 2.9M patents(14.4GB) with consistent performance
- Linear cost scaling through partition pruning(84% data reduction for targeted searches)
- Architecture supports horizontal scaling to full 49M patent corpus via month-based partitioning

**Query Performance Scalability:**
- Achieves sub-4 second vector search response times across 2.9M patents with partition pruning enabling cost-effective scaling to larger datasets
- Consistent performance regardless of partition size (2.8-3.9 seconds for vector search operations)
- 1-month partitioning delivers 12% performance improvement in addition to 84% cost reduction
- Partition pruning demonstrates stable performance despite 5x variation in data volume

**Cost-Efficient Scalability:**
- Partition pruning delivers 84% reduction in bytes processed for 1-month searches
- Production deployment cost optimization through BigQuery clustering strategies
- Enterprise+ tier compatibility enables vector indexing for further performance gains

**Concurrent Processing Scalability:**
- Batch processing architecture handles 5,000-record embedding generation efficiently
- Multi-environment deployment validation (Kaggle GPU vs local processing)
- Separation of offline bulk processing (embeddings) from real-time queries enables concurrent user support


In [ ]:
create_bigquery_visualization()

#### Performance Testing Results

**Latency Measurements:** **Environment Comparison**

Core Vector Search Performance<br>
**Kaggle Environment:**

- Mean latency: 3.0 seconds
- Median latency: 2.9 seconds
- 90th percentile: 3.2 seconds
- Range: 2.5s - 4.2s

**Laptop Environment:**

- Mean latency: 4.5 seconds
- Median latency: 4.2 seconds
- 90th percentile: 4.9 seconds
- Range: 3.9s - 7.2s

Complete Pipeline Performance (with Explainability)<br>
**Kaggle Environment:**

- Mean latency: 5.2 seconds
- Median latency: 5.0 seconds
- 90th percentile: 5.5 seconds
- Range: 4.2s - 6.7s

**Laptop Environment:**

- Mean latency: 12.9 seconds
- Median latency: 11.6 seconds
- 90th percentile: 15.2 seconds
- Range: 9.4s - 18.7s

Key Performance Insights

**Environment Impact:** Kaggle environment shows 33% faster vector search and 60% faster complete pipeline performance, highlighting the benefits of cloud-native deployment proximity to BigQuery infrastructure.<br>
**Consistency:** Kaggle environment demonstrates lower variability (std dev: 467ms vs 991ms for vector search), indicating more predictable performance characteristics for production deployment.<br>
**Production Scalability:** Both environments successfully process semantic searches across 2.9M patents, with vector search completing in under 5 seconds across all test scenarios.

In [ ]:
create_latency_visualization()

#### Semantic versus Keyword Search Discovery Analysis

**Discovery Methodology:** Comparative analysis across 10 diverse technology queries that evaluate the discovery capabilities between BigQuery AI semantic search and traditional keyword search approaches. Semantic search uniqueness percentage represents the percentage of semantic search results that were not returned by keyword search methods. Each query was executed, to return a maximum of 30 results,  across both latop and Kaggle environments to validate consistency. <br>
The testing revealed significant differences in discovery capabilities between semantic and keyword search approaches.<br>

Note - Without a ground truth dataset, we cannot accurately measure recall or precision

Search Performance Comparison<br>
**Keyword Search:**

Average results per query: 18 patents
Average search time: 2.1 seconds (Kaggle), 3.4 seconds (laptop)

**Semantic Search:**

Average results per query: 30 patents
Average search time: 2.7 seconds (Kaggle), 5.0 seconds (laptop)

**Discovery Effectiveness:**
**Cross-Environment Validation:** Testing across both laptop and Kaggle environments confirmed consistent discovery capabilities, with semantic search finding 295 unique patents and keyword search finding 175 unique patents in both environments:

- **Total semantic results:** 300 patents (30 per query × 10 queries)
- **Total keyword results:** 180 patents (18 per query × 10 queries)

**Unique Patent Discovery:**

- Semantic search discovers 295 patents not found by keyword search
- Keyword search discovers 175 patents not found by semantic search
- Overlap: 5 patents found by both methods

Search Method Limitations<br>
**Keyword Search Challenges:**

- Complex, multi-term technical queries often return zero results
- Long descriptive queries (e.g., "quantum error correction topological qubits fault tolerance") fail to match patent terminology
Requires exact or near-exact term matching, missing conceptually similar descriptions

**Semantic Search Robustness:**

- Handles complex technical descriptions effectively
- Finds conceptually related patents even with different terminology
- Maintains consistent result quality across diverse query complexity levels

**Implications for Patent Research**

**Semantic Search Superiority:** Semantic search successfully finds patents (relevant or not cannot be determined with a subject matter expert or a ground truth dataset) for all query types, including complex technical descriptions where keyword search returns zero results.<br>
**Complementary Discovery:** The minimal overlap (5 patents) demonstrates that semantic and keyword searches access fundamentally different types of patent relevance, with semantic search providing broader conceptual coverage.<br>
**Query Complexity Handling:** Semantic search succeeds where keyword search fails on complex technical descriptions, making it essential for sophisticated patent research workflows.



In [ ]:
create_discoverability_visualization()

## Technical Challenges

#### Embedding Generation Performance Bottleneck


**Initial ML.GENERATE_EMBEDDING Challenges:**
The biggest technical hurdle was BigQuery's native embedding generation performance. Initial attempts using ML.GENERATE_EMBEDDING on 2013-2014 patent data revealed severe limitations:
- **Monitoring Issues:** Jobs ran for 30+ minutes with no meaningful progress tracking beyond basic UI status
- **Performance Variability:** Batch processing showed extreme variation (7-70 minutes per 5,000 record batch)
- **Scale Reality:** Processing 48K records (one day) took 6 hours across 10 batches, averaging 36 minutes per batch
- **Cost-Performance Trade-off:** At ~0.45 seconds per embedding, processing 2.9M patents would require 15 days

**Implementation Iterations:**

**First attempt:** Single large job (2013-2014) with no progress tracking - abandoned after 30 minutes<br>
**Second iteration:** Stored procedure with 2,000 record batches - still too slow with no visibility<br>
**Third iteration:** Added BigQuery logging, increased to 5,000 record batches, reduced scope to 1 day (48K); the job completed in 6 hours<br>
**Data Quality Issue:** Initial approach using LIMIT + OFFSET without ORDER BY created duplicates; resolved with ROW_NUMBER() windowing

**Kaggle GPU Solution:**
The breakthrough came with Sentence Transformers on Kaggle GPU, processing 2.9M records (14GB) in 6.5 hours using 5,000-record batches - a dramatic improvement that made the project feasible.

#### BigQuery Edition Limitations

**Vector Index Discovery:**
After implementing comprehensive vector indexing with 100% coverage on the 14.4GB dataset, we discovered that BigQuery Standard edition (free tier) supports VECTOR_SEARCH creation functionality but restricts vector index utilization to Enterprise+ tiers. This limitation highlights critical production planning considerations for BigQuery AI deployments.

#### Development Environment Challenges

**Kaggle-Specific Issues:**

**Dependency conflicts:** Library version mismatches requiring minimal dependency strategies<br>
**Repository integration:** Inconsistent import paths (/kaggle/input vs /kaggle/input/d/username) requiring defensive coding<br>
**Workflow friction:** Manual repository re-import for each code change as the 'Link To GitHub' feature wasn't automatically picking up changes<br>
**Environment incompatibility:** Kaggle's environment lacks Streamlit support, forcing maintenance of separate requirements.txt files divergent from the project's pyproject.toml, including charts and demos - violating software engineering principles of single source of truth and creating dual repository management overhead

**Solution Strategy:** Minimize external dependencies and leverage Kaggle's pre-installed libraries to avoid conflicts, while accepting the technical debt of maintaining divergent dependency specifications.

## Assets

1. [bigquery_ai_survey](https://www.kaggle.com/datasets/laxmsun/bigquery-ai-survey)
2. [patent_art_github](https://github.com/sl2902/patent_art)
3. [streamlit demo](https://patent-art.streamlit.app/)

In [ ]:
# import generate_patent_embeddings
# set the env variable os.environ["embedding_table"] = "table_name"

In [ ]:
# !python generate_patent_embeddings.py --date-start='2017-01-02' --date-end='2017-01-02' --batch-size=1000